In [1]:
import numpy as np
import pandas as pd
import os
from IPython.display import clear_output
import time

import sys

# Path to library directory.
_lib_path = os.path.join('c:' + os.sep, 'Users', 'natha', 'OneDrive', 'Documents', 'GitHub', 'Debiasing_Kepler_Data', 'lib')

# Allows access to modules in the library directory.
if _lib_path not in sys.path:
    sys.path.append(_lib_path)

import system_names
import index_combinations

In [2]:
VERBOSE = True
USE_ALL_DATA = False

In [3]:
def generate_mutual_inclination_data(savepath_directory, system_filepaths):
    if type(system_filepaths) != list:
        raise Exception('Please input a list of system filepaths! Type of input is not a list.')

    try:
        # Loops through all the systems in the given filepaths.
        for system_filepath in system_filepaths:
            # Gets the system name from the end of the filepath.
            system_name = system_filepath.split(os.sep)[-1]

            if VERBOSE:
                seconds = time.time()
                
                if (seconds % 1.0) < 0.08:
                    clear_output()
                    print('Caching data from {system}...({pct}%)'.format(system = system_name, pct = round(((system_filepaths.index(system_filepath) + 1) / len(system_filepaths) * 100.0))))

            # Accesses the data as a pandas dataframe.
            data = pd.read_csv(system_filepath)

            # Gets the final timestep of the dataset.
            endtime = data['Time'].iloc[-1].astype(float)

            # Checks if the given file integrated to 1 Myr.
            if (not USE_ALL_DATA) and endtime < 1e6:
                continue

            # Creates data frame with first column as the time column.
            mutual_inclinations_data_frame = pd.DataFrame({'Time': data['Time']})

            # Gets the columns which contain inclination data.
            inclination_columns = sorted([column for column in data.columns if 'inclination' in column])
            Omega_columns = sorted([column for column in data.columns if 'Omega' in column])

            # Ensures the arrays are of equal length to prevent indexing errors.
            assert len(inclination_columns) == len(Omega_columns)

            # Gets the multiplicity of the system. There should be as many planets as there are unique inclination columns.
            # I.e. there is "Planet 1 inclination", "Planet 2 inclination", ... "Planet N inclination", where N is the system
            # multiplicity.
            multiplicity = len(inclination_columns)

            # Loops through every planet-planet pair combination for the given system.
            for (n1, n2) in index_combinations.get_index_combinations(multiplicity):                
                # Ensures that the indexes correspond to the correct columns. Makes sure the order is correct.
                assert str(n1 + 1) in inclination_columns[n1]
                assert str(n2 + 1) in inclination_columns[n2]
                
                # Captures the inclination pair data, converting from degrees to radians.
                n1_inclinations = (data[inclination_columns[n1]].astype(float))
                n2_inclinations = (data[inclination_columns[n2]].astype(float))
                n1_Omegas = (data[Omega_columns[n1]].astype(float))
                n2_Omegas = (data[Omega_columns[n2]].astype(float))

                # Calculates the mutual inclinations, converting from radians to degrees.
                mutual_inclinations = np.rad2deg(np.arccos(np.cos(n1_inclinations)*np.cos(n2_inclinations) + np.sin(n1_inclinations)*np.sin(n2_inclinations)*np.cos(n1_Omegas-n2_Omegas)))

                # Creates a label for the planet pair.
                planet_pair = 'Planets {planet_1} and {planet_2}'.format(planet_1 = (n1 + 1), planet_2 = (n2 + 1))
                
                column_label = planet_pair + ' mutual inclination'

                # Writes the data to the dataframe.
                mutual_inclinations_data_frame[column_label] = mutual_inclinations

            savepath = os.path.join(savepath_directory, system_name)

            mutual_inclinations_data_frame.to_csv(savepath, index = False)

        # Lets the user know the data caching has completed.
        if VERBOSE:
            print('Done!')

    except FileNotFoundError as exception:
        print(exception)

In [4]:
system_names = [
    'sysSim_1.csv', 'sysSim_1_randomO.csv', # 1
    'sysSim_7.csv', 'sysSim_7_randomO.csv', # 2
    'sysSim_25.csv', 'sysSim_25_randomO.csv', # 3
    'sysSim_77.csv', 'sysSim_77_randomO.csv', # 4
    'sysSim_27.csv', 'sysSim_27_randomO.csv', # 5
    'sysSim_18.csv', 'sysSim_18_randomO.csv', # 6
    'sysSim_186.csv', 'sysSim_186_randomO.csv', # 7
    'sysSim_151.csv', 'sysSim_151_randomO.csv', # 8
    'sysSim_422.csv', 'sysSim_422_randomO.csv', # 9
    'sysSim_169.csv', 'sysSim_169_randomO.csv', # 10
]

# Generates the filepaths given the system names.
system_filepaths = [os.path.join(os.pardir, os.pardir, 'data', 'raw', system_name) for system_name in system_names]

savepath_directory = os.path.join(os.pardir, os.pardir, 'data', 'processed')

generate_mutual_inclination_data(savepath_directory, system_filepaths)

Caching data from sysSim_18_randomO.csv...(60%)
Done!


In [5]:
def find_fully_integrated_systems(start, end):
    fully_integrated = list()
    
    systems = system_names.system_names[start:end]

    # Generates the filepaths given the system names.
    system_filepaths = [os.path.join(os.pardir, os.pardir, 'data', 'raw', system_name) for system_name in systems]

    for system_filepath in system_filepaths:
        file = open(system_filepath, 'rb')

        # Seeks past the first return characters.
        pos = -2

        # Seeks past the first return characters.
        file.seek(pos, os.SEEK_END)

        while file.read(1).decode() != '\n':
            pos -= 1
            file.seek(pos, os.SEEK_END)

        endtime = float(file.readline().decode()[:25].split(',')[0])

        file.close()

        if endtime == 1e6:
            fully_integrated.append(system_filepath)
    
    return fully_integrated

def find_systems_of_multiplicity(multiplicity, full_integration = False):
    matches = list()

    systems = system_names.system_names

    # Generates the filepaths given the system names.
    system_filepaths = [os.path.join(os.pardir, os.pardir, 'data', 'raw', system_name) for system_name in systems]

    for system_filepath in system_filepaths:
        file = open(system_filepath, 'rb')

        header = file.readline().decode()

        system_multiplicity = int(header[-11])

        if system_multiplicity != multiplicity:
            continue
        
        # Seeks past the first return characters.
        pos = -2

        # Seeks past the first return characters.
        file.seek(pos, os.SEEK_END)

        while file.read(1).decode() != '\n':
            pos -= 1
            file.seek(pos, os.SEEK_END)

        endtime = float(file.readline().decode()[:40].split(',')[0])

        file.close()

        if full_integration and endtime < 1e6:
            continue

        matches.append(system_filepath)
    
    return matches

In [6]:
find_systems_of_multiplicity(2, full_integration = True)

AttributeError: 'list' object has no attribute 'system_names'